# Purpose
Check whether GPUs are available/usable in current python environment.


# Debugging notes
## Make sure to set the correct venv/kernel in your notebook
The default `python 3` might not have the correct drivers.
<br>Instead, might need to manually set it to:
<br>`Python [conda env:root]`

## Sometimes the best fix is `sudo reboot`
When in doubt, open a terminal and do `sudo reboot`.

For some reason, the NVIDIA drivers might not be loaded properly after shutting down a VM instance from the GUI:
- https://console.cloud.google.com/ai-platform/notebooks/list/instances?project=data-prod-165221


In [1]:
!which python

/opt/conda/bin/python


# Imports & notebook setup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pprint import pprint

import tensorflow_text
import tensorflow as tf
from tensorflow.python.client import device_lib

import subclu
from subclu.utils.eda import (
    setup_logging, notebook_display_config, print_lib_versions,
)


print_lib_versions([tf, tensorflow_text, subclu])

python		v 3.7.10
===
tensorflow	v: 2.3.2
tensorflow_text	v: 2.3.0
subclu		v: 0.1.1


In [4]:
setup_logging()

# Check GPUs recognized by Tensorflow/python

NOTE: `GPU`s and `XLA_GPU`s are recognized as two different device types.

https://www.tensorflow.org/xla
> **XLA: Optimizing Compiler for Machine Learning**
> XLA (Accelerated Linear Algebra) is a domain-specific compiler for linear algebra that can accelerate TensorFlow models with potentially no source code changes.
> 
> The results are improvements in speed and memory usage: e.g. in BERT MLPerf submission using 8 Volta V100 GPUs using XLA has achieved a ~7x performance improvement and ~5x batch size improvement

Other sources
- https://stackoverflow.com/questions/52943489/what-is-xla-gpu-and-xla-cpu-for-tensorflow


In [10]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [12]:
%%time

tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
CPU times: user 1.39 ms, sys: 1.23 ms, total: 2.61 ms
Wall time: 1.84 ms


In [5]:
l_phys_gpus = (
    tf.config.list_physical_devices('GPU') +
    tf.config.list_physical_devices('XLA_GPU')
)

print(
    f"\nBuilt with CUDA? {tf.test.is_built_with_cuda()}"
    f"\n\nGPUs\n==="
    f"\nNum GPUs Available: {len(l_phys_gpus)}"
    f"\nGPU details:\n{l_phys_gpus}"
)


Built with CUDA? True

GPUs
===
Num GPUs Available: 0
GPU details:
[]


In [6]:
l_all_local_devices = device_lib.list_local_devices()
print(
    f"\nBuilt with CUDA? {tf.test.is_built_with_cuda()}"
    f"\n\nAll devices:\n==="
    f"\nNum devices: {len(l_all_local_devices)}"
    f"\nDetails:"
)
pprint(l_all_local_devices, indent=4,)


Built with CUDA? True

All devices:
===
Num devices: 2
Details:
[   name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4153781128868626988
,
    name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1204199666303421190
physical_device_desc: "device: XLA_CPU device"
]


# Check NVIDIA CLI

First, do we even see the GPUs?

In [7]:
!lspci | grep 3D

00:04.0 3D controller: NVIDIA Corporation TU104GL [Tesla T4] (rev a1)
00:05.0 3D controller: NVIDIA Corporation TU104GL [Tesla T4] (rev a1)


Then, are they recognized by the nvidia-smi tool?

In [8]:
!nvidia-smi

Wed Jun 30 04:11:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    31W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

## Debug nvidia drivers

If `nvidia-smi` doesn't detect the drivers, we might need to reinstall them.

- https://towardsdatascience.com/troubleshooting-gcp-cuda-nvidia-docker-and-keeping-it-running-d5c8b34b6a4c

Getting nothing from `cuda` is bad... sigh

In [9]:
!dpkg -l | grep nvidia

ii  libnvidia-container-tools             1.4.0-1                       amd64        NVIDIA container runtime library (command-line tools)
ii  libnvidia-container1:amd64            1.4.0-1                       amd64        NVIDIA container runtime library
ii  nvidia-container-runtime              3.5.0-1                       amd64        NVIDIA container runtime
ii  nvidia-container-toolkit              1.5.1-1                       amd64        NVIDIA container runtime hook
ii  nvidia-docker2                        2.6.0-1                       all          nvidia-docker CLI wrapper


In [12]:
!dpkg -l | grep cuda

In [10]:
!dmesg | grep NVIDIA

dmesg: read kernel buffer failed: Operation not permitted


In [11]:
!apt search nvidia-driver

Sorting... Done
Full Text Search... Done
